In [7]:
import json

from ipyleaflet import Map, LocalTileLayer, basemap_to_tiles, basemaps
from ipyleaflet import LayersControl, ZoomControl, ScaleControl, LegendControl, WidgetControl, DrawControl
import IPython.display
import ipywidgets as widgets


with open('stats.json') as stats_json:
    stats = json.load(stats_json)
    
minimum = stats['minimum']
maximum = stats['maximum']


chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'

m = Map(
    basemap=chosen_basemap,
    center=(0,0),
    zoom=2,  # zoom level for most of the globedisplay(m)
)  

scalebar = ScaleControl(position='bottomleft')
m.add_control(scalebar)

# If we want to pre-cache these tiles on GCS, use ipyleaflet.leaflet.TileLayer instead
country_id_tiles_layer = LocalTileLayer(
    path='tiles/{z}/{x}/{y}.png',
    name='NASA SRTM',
    show_loading=True,
    attribution='NASA'
    
)
m.add_layer(country_id_tiles_layer)

layerlist = LayersControl(position='topleft')
m.add_control(layerlist)

# build a colormap from our GDAL-compatible GRASS color file
import branca
colors = []
with open('elevation-color-profile-grass.txt') as color_profile:
    for line in color_profile:
        try:
            percent, r, g, b = [c.strip() for c in line.split(' ') if c]
        except ValueError:
            pass
        colors.append((float(r), float(g), float(b)))

colormap = branca.colormap.LinearColormap(
    colors=colors,
    vmin=minimum, vmax=maximum)
colormap.caption = 'Elevation (m)'
html = f"{colormap._repr_html_()}"

colorbar = WidgetControl(widget=widgets.HTML(html), position='topright')
m.add_control(colorbar)


draw_control = DrawControl(polyline={}, circlemarker={})
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 0.5
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}
m.add_control(draw_control)

m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [8]:
draw_control.data[0]['geometry']

IndexError: list index out of range

In [9]:
import shapely.geometry
shapely.geometry.Polygon(draw_control.data[0]['geometry']['coordinates'][0])

IndexError: list index out of range

In [11]:
# How to get layer names passed in as URL parameters.
# 
# A button in the catalog can be programmatically built in this way
# to link to this jupyter notebook wherever it is served and have
# the notebook constructed so that we pre-select the target layer.
#
# https://stackoverflow.com/a/37134476

from IPython.display import HTML
HTML('''
    <script type="text/javascript">
        IPython.notebook.kernel.execute("URL = '" + window.location + "'")
    </script>''')
print(URL)

NameError: name 'URL' is not defined